# **LP2_Run3**

In [2]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [3]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """You are in charge of the supply purchasing of a company that produces two kinds of drugs.
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market.

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales.

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded.
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available.
Your company can also only store a limited amount of raw materials.
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces.
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively.
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met.  """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

Parameters:
1. R = {1, 2}, index set of raw materials.
2. C_r : Cost per unit of raw materials r.
3. B : Total budget for purchasing raw materials and operating the production process.
4. E_r : Maximum extraction rate of active ingredient from raw material r.
5. F_r : Fluctuation rate of extraction of active ingredient from raw material r.
6. D : total demand for the active agent.
7. M : Man power available.
8. H : Equipment hours available.
9. S_r : storage capacity of raw material r.
10. O : Operational cost per unit of drug produced.
11. P : Price per unit of drug sold.

Decision Variables:
1. x_r : Amount of raw material r to be purchased.
2. y : Amount of drugs to be produced.

Objective Function:
Maximize Z = P*y - Σ_r C_r*x_r - O*y. The aim is to maximize total profit.

Constraints:
1. Budget constraint: Σ_r C_r*x_r + O*y ≤ B.
2. Manpower and hours constraint: Manpower required * y ≤ M and Equipment hours required * y ≤ H (let's say every unit of drug requires 1 unit of manpower

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
Markdown(response2.choices[0].message.content)

Here is the python pyomo code for the given optimization problem.

SAMPLE DATA USED:
1. R = ['Raw1', 'Raw2']
2. Cost of Raw1 = $40
3. Cost of Raw2 = $70
4. Extract rates = {'Raw1': 0.5, 'Raw2': 0.4}
5. Fluctuation rate = {'Raw1': 5, 'Raw2': 10}
6. Budget = $10000
7. Demand = 10 units
8. Man power available = 500
9. Equipment hours available = 1000
10. Storage capacity = {'Raw1': 100, 'Raw2': 200}
11. Operational cost = $30
12. Price = $100

```python
from pyomo.environ import *

# Sample data
R = ['Raw1', 'Raw2']
C_r = {'Raw1': 40, 'Raw2': 70}
B = 10000
E_r = {'Raw1': 0.5, 'Raw2': 0.4}
F_r = {'Raw1': 5, 'Raw2': 10}
D = 10
M = 500
H = 1000
S_r = {'Raw1': 100, 'Raw2': 200}
O = 30
P = 100

# Create the model
model = ConcreteModel()

# Decision variables
model.x_r = Var(R, domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# Objective function
model.profit = Objective(expr=P*model.y - sum(C_r[r]*model.x_r[r] for r in R) - O*model.y, sense=maximize)

# Constraints
model.budget = Constraint(expr=sum(C_r[r]*model.x_r[r] for r in R) + O*model.y <= B)
model.man_hour = Constraint(expr=model.y <= M and model.y <= H)
model.storage = ConstraintList()
for r in R:
    model.storage.add(expr=model.x_r[r] <= S_r[r])
model.ingredient = Constraint(expr=sum(E_r[r]*model.x_r[r]*(1-F_r[r]/100) for r in R) >= D)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
for r in R:
    print(f"Amount of raw material {r} to be purchased: {model.x_r[r]()}")
print(f"Amount of drugs to be produced: {model.y()}")
print(f"Total Profit: {model.profit()}")
```
Please note that the above code requires installation of Pyomo and GLPK solver. The results of the computation would be displayed when you run the script.

### **Run the code Generated by GPT4**

In [13]:
from pyomo.environ import *

# Sample data - written by human
R = ['Raw1', 'Raw2']
C_r = {'Raw1': 100, 'Raw2': 199.90}
B = 100000
E_r = {'Raw1': 0.5, 'Raw2': 0.4} #not in correct model
F_r = {'Raw1': 0.005, 'Raw2': 0.2}
D = 10 #not in correct model
M = 2000
H = 800
S_r = {'Raw1': 500, 'Raw2': 500} #should be 100 and 1 capacity - incorrect parameter.
O = 750 # should have been 2 seperate operational costs.
P = 6500 # average of the twp, as there should have been 2 prices for 2 drugs.
#END

# Create the model
model = ConcreteModel()

# Decision variables
model.x_r = Var(R, domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# Objective function
model.profit = Objective(expr=P*model.y - sum(C_r[r]*model.x_r[r] for r in R) - O*model.y, sense=maximize)

# Constraints
model.budget = Constraint(expr=sum(C_r[r]*model.x_r[r] for r in R) + O*model.y <= B)
model.man_hour = Constraint(expr=model.y <= M and model.y <= H)
model.storage = ConstraintList()
for r in R:
    model.storage.add(expr=model.x_r[r] <= S_r[r])
model.ingredient = Constraint(expr=sum(E_r[r]*model.x_r[r]*(1-F_r[r]/100) for r in R) >= D)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
for r in R:
    print(f"Amount of raw material {r} to be purchased: {model.x_r[r]()}")
print(f"Amount of drugs to be produced: {model.y()}")
print(f"Total Profit: {model.profit()}")

PyomoException: Cannot convert non-constant Pyomo expression (y  <=  2000) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [10]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, SolverFactory, summation, maximize

# Create a model
model = ConcreteModel()

# Sets
model.R = ['Raw1', 'Raw2']  # Set of raw materials

# Parameters
C = {'Raw1': 40, 'Raw2': 70}  # Cost per unit of raw materials
B = 10000  # Total budget
E = {'Raw1': 0.5, 'Raw2': 0.4}  # Maximum extraction rate of active ingredient
F = {'Raw1': 5, 'Raw2': 10}  # Fluctuation rate of extraction
D = 10  # Total demand for the active agent
M = 500  # Manpower available
H = 1000  # Equipment hours available
S = {'Raw1': 100, 'Raw2': 200}  # Storage capacity of raw materials
O = 30  # Operational cost per unit of drug produced
P = 100  # Price per unit of drug sold

# Decision Variables
model.x = Var(model.R, domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# Objective Function
def objective_rule(model):
    return P * model.y - summation(C, model.x) - O * model.y

model.Profit = Objective(rule=objective_rule, sense=maximize)

# Constraints

# Budget constraint
def budget_constraint(model):
    return summation(C, model.x) + O * model.y <= B

model.BudgetConstraint = Constraint(rule=budget_constraint)

# Manpower and equipment hours constraints
def manpower_constraint(model):
    return model.y*95 <= M

def equipment_hours_constraint(model):
    return model.y*45 <= H
#means of the manpower and equipment requierments were taken for drug 1 and 2

model.ManpowerConstraint = Constraint(rule=manpower_constraint)
model.EquipmentHoursConstraint = Constraint(rule=equipment_hours_constraint)

# Storage constraints
def storage_constraint_rule(model, r):
    return model.x[r] <= S[r]

model.StorageConstraints = Constraint(model.R, rule=storage_constraint_rule)

# Active ingredient constraint
def active_ingredient_constraint(model):
    return sum(E[r] * model.x[r] * (1 - F[r] / 100) for r in model.R) >= D

model.ActiveIngredientConstraint = Constraint(rule=active_ingredient_constraint)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

# Display the results
model.display()


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpj4mu3ppy.glpk.raw --wglp /tmp/tmpqxabgldg.glpk.glp --cpxlp
 /tmp/tmpntx6t3zl.pyomo.lp
Reading problem data from '/tmp/tmpntx6t3zl.pyomo.lp'...
6 rows, 3 columns, 9 non-zeros
42 lines were read
Writing problem data to '/tmp/tmpqxabgldg.glpk.glp'...
30 lines were written
GLPK Simplex Optimizer 5.0
6 rows, 3 columns, 9 non-zeros
Preprocessing...
2 rows, 3 columns, 5 non-zeros
Scaling...
 A: min|aij| =  3.600e-01  max|aij| =  7.000e+01  ratio =  1.944e+02
GM: min|aij| =  8.112e-01  max|aij| =  1.233e+00  ratio =  1.520e+00
EQ: min|aij| =  6.581e-01  max|aij| =  1.000e+00  ratio =  1.520e+00
Constructing initial basis...
Size of triangular part is 2
      0: obj =  -0.000000000e+00 inf =   2.108e+01 (1)
      1: obj =  -8.421052632e+02 inf =   0.000e+00 (0)
*     2: obj =  -4.736842105e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40412 bytes)
Writing 